<a href="https://colab.research.google.com/github/pjd-code/HITS_microbiome/blob/main/Qiime2_import_demultiplexed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#This block of code downloads and installs conda and qiime2

#all outputs are directed to a log file using the bash pipe operator
#updated versions of qiime2 or miniconda can be substituted

#download and install miniconda
! wget https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh &> Logfile.txt
! chmod +x Miniconda3-4.5.4-Linux-x86_64.sh &>> Logfile.txt
! bash ./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local &>> Logfile.txt
print("1. Conda Installed!")

#import the python system library and append the path
import sys
(sys.path.append("/usr/local/lib/python3.6/site-packages"))

#update conda while keeping python 3.6
! conda install --channel defaults conda python=3.6 --yes &>> Logfile.txt
! conda update --channel defaults --all --yes &>> Logfile.txt
print ("\n2. Conda Updated!")

#The instructions for this step come from https://docs.qiime2.org/2021.4/install/native/
! wget https://data.qiime2.org/distro/core/qiime2-2021.4-py38-linux-conda.yml &>> Logfile.txt
! conda env create -n qiime2-2021.4 --file qiime2-2021.4-py38-linux-conda.yml &>> Logfile.txt
#cleanup
! rm qiime2-2021.4-py36-linux-conda.yml &>> Logfile.txt

! conda init &>> Logfile.txt
print("\n3. Conda Initiated!")
print("\n4. Qiime2 Install Complete! All Qiime2 cells must start with:  source activate qiime2-2021.4")

In [ ]:
#mount the hitsmicrobime google drive so that we can get the relevant files
#this is python code so it doesn't need the !

from google.colab import drive
drive.mount('/content/drive')



In [ ]:
#the %%bash magic command makes the entire cell a bash command line cell
%%bash 

#make a directory for the imported files
mkdir /content/microbiome

#move map file to microbime folder for future use
cp /content/drive/MyDrive/Keto_Mice/map_file.txt /content/microbiome/

source activate qiime2-2021.2 

qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \
  --input-path /content/microbiome/map_file.txt \
  --output-path /content/microbiome/paired-end-demux.qza \
  --input-format PairedEndFastqManifestPhred33V2

#supporting information can be found at https://docs.qiime2.org/2021.4/tutorials/importing/

In [ ]:
%%bash
source activate qiime2-2021.4

cd /content/microbiome/

qiime demux summarize \
  --i-data paired-end-demux.qza \
  --o-visualization paired-end-demux

In [ ]:
%%bash
source activate qiime2-2021.4

cd /content/microbiome/

# This method denoises paired-end sequences, dereplicates them, and filters chimeras.
# p-trunc-len-f : truncates the 3' end of the forward read sequences due to decrease in quality
# p-trunc-len-r : truncates the 3' end of the reverse read sequences due to decrease in quality
# p-trim-left-f : trims the 5' end of the forward read sequences due to decrease in quality
# p-trim-left-r  : trims the 5' end of the reverse read sequences due to decrease in quality

qiime dada2 denoise-paired \
  --i-demultiplexed-seqs paired-end-demux.qza \
  --p-trunc-len-f 250 \
  --p-trunc-len-r 140 \
  --p-trim-left-f 0 \
  --p-trim-left-r 0 \
  --o-representative-sequences rep-seqs-dada2.qza \
  --o-table table-dada2.qza \
  --o-denoising-stats stats-dada2.qza

In [ ]:
%%bash
source activate qiime2-2021.4

cd /content/microbiome/

qiime metadata tabulate \
  --m-input-file stats-dada2.qza \
  --o-visualization stats-dada2.qzv



In [ ]:
%%bash
source activate qiime2-2021.4

cd /content/microbiome/

qiime phylogeny align-to-tree-mafft-fasttree \
  --i-sequences rep-seqs-dada2.qza \
  --o-alignment aligned-rep-seqs.qza \
  --o-masked-alignment masked-aligned-rep-seqs.qza \
  --o-tree unrooted-tree.qza \
  --o-rooted-tree rooted-tree.qza

In [ ]:
%%bash
source activate qiime2-2021.4

cd /content/microbiome/

qiime diversity core-metrics-phylogenetic \
  --i-phylogeny rooted-tree.qza \
  --i-table table-dada2.qza \
  --p-sampling-depth 2000 \
  --m-metadata-file map_file.txt \
  --output-dir core-metrics-results

In [ ]:
%%bash
source activate qiime2-2021.4

cd /content/microbiome/

#wget \
#  -O "gg-13-8-99-515-806-nb-classifier.qza" \
 # "https://data.qiime2.org/2021.4/common/gg-13-8-99-nb-classifier.qza"

  qiime feature-classifier classify-sklearn \
  --i-classifier gg-13-8-99-515-806-nb-classifier.qza \
  --i-reads rep-seqs-dada2.qza \
  --o-classification taxonomy.qza



In [ ]:
%%bash
source activate qiime2-2021.4

cd /content/microbiome/

qiime metadata tabulate \
  --m-input-file taxonomy.qza \
  --o-visualization taxonomy.qzv


In [ ]:

%%bash
source activate qiime2-2021.4

cd /content/microbiome/
qiime taxa barplot \
  --i-table table-dada2.qza \
  --i-taxonomy taxonomy.qza \
  --m-metadata-file map_file.txt \
  --o-visualization taxa-bar-plots.qzv